# Apply small color CNN to all patches

Use the patches created in DF_Filter.018  
These are 'mixed' patches i.e. not obviously blue, green, or black.
Use the CNN model trained on 3K patches in DF_Filter.017 (didn't use data augmentation).    
Be careful to call cnn.predict_classes() not cnn.compile() or cnn.fit().  
Remember to apply pixel scaling.  
Use keras.preprocessing (https://keras.io/examples/vision/image_classification_from_scratch/).  

In [1]:
import time
import os
#  os.environ['CUDA_VISIBLE_DEVICES'] = "0"   # turns off GPU?
import glob
import cv2 # OpenCV-Python
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import numpy as np
import json
import tensorflow as tf
from tensorflow import keras
import keras.layers as kl
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
DIR_IMAGES_IN = "/home/jrm/Martinez/images/patches/"
DIR_IMAGES_IN = "/home/jrm/Martinez/images/tinytest/"   # just for testing
PATTERN_PATCHES = "*.jpg"
PATCH_SIZE=224  # matches VGG
IMAGE_SIZE=(PATCH_SIZE,PATCH_SIZE)
DIR_MODELS = "/home/jrm/Martinez/models/"
FILE_MODEL = "CNN_SMALL_COLOR"
filepath=DIR_MODELS+FILE_MODEL
print("Will use model from "+filepath)
cnn = keras.models.load_model(filepath)

Will use model from /home/jrm/Martinez/models/CNN_SMALL_COLOR


In [3]:
def get_image_names(path,pattern):
    paths = glob.glob(path+pattern)
    names = [os.path.basename(x) for x in paths]
    return names
FILENAMES_PATCHES = get_image_names(DIR_IMAGES_IN,PATTERN_PATCHES)

In [4]:
normalization_layer = kl.Rescaling(1.0/255.0)

In [7]:
# With keras models, augmentation and dropout are inactive during prediction.
start = time.time()
for filename in FILENAMES_PATCHES:
    path = DIR_IMAGES_IN+filename
    img = keras.preprocessing.image.load_img(
        path, target_size=IMAGE_SIZE
    )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = normalization_layer(img_array)
    img_array = tf.expand_dims(img_array, 0)  # create tensor
    # TODO: consider using predict_classes() instead
    predictions = cnn.predict(img_array)
    score = predictions[0]  # TODO: explore other array slices
    print(
        "Image %s cores %f blue and %f green."
        % (filename, score[0], score[1])
    )
end = time.time()
print("Elapsed time:",end-start)

Image GREEN.D3.250.jpg cores 0.000179 blue and 0.999821 green.
Image BLUE.E5.100.jpg cores 0.999842 blue and 0.000158 green.
Elapsed time: 0.06403684616088867
